# Deep learning

## 1. Introduction

Examples and code for explaining machine learning and deep learning concepts.

This tutorial has been prepared by:
- [Andrew Rohl](http://computation.curtin.edu.au/about/steering-committee/director/)
- [Shiv Meka](http://computation.curtin.edu.au/about/computational-specialists/humanities/)
- [Kevin Chai](http://computation.curtin.edu.au/about/computational-specialists/health-sciences/)

from the [Curtin Institute for Computation](http://computation.curtin.edu.au) at Curtin University in Perth, Australia for the [7th International Conference on Smart Computing & Communications (ICSCC 2019)](http://icscc.online/) hosted at Curtin University in Miri, Sarawak, Malaysia on the 28-30 June 2019.

## 2. Installation

Install the necessary python dependencies. Some packages marked with an __^__ are available on <a href='https://www.anaconda.com/distribution/' target='_blank'>Anaconda's Python </a>distribution by default and for the others you can use a package installer - `pip` or `easy_install`: 

- Python 3 ^
- jupyter ^
- numpy ^
- scipy ^
- PIL ^
- tarfile ^
- requests ^
- tensorflow
- keras
- plotly

In [ ]:
# Check installed packages
!pip list

Import the libraries needed for the notebook.

In [ ]:
# Import all essential subroutines for data processing and visualization
from workshop import * 

# Plotting libraries
import plotly.graph_objs as go 
from plotly.offline import plot, iplot, init_notebook_mode 
import matplotlib.pyplot as PyPlot

# Scientific library needed for most exercises
import numpy as np

# Machine learning functions
from keras.layers import Input, Dense
from keras.models import Model

# Plot graphs in the notebook itself w/o opening a new window
init_notebook_mode(True)
%matplotlib inline 

## 3. Curve fitting

In [ ]:
# Display slides for the section
PDF('assets/exercise1.pdf', 3, 600, 800)

### 3.1 Measure distance / loss

In [ ]:
# Mean-squared distance
def mse(inp, pred):
    '''
    Mean-squared error (MSE) amplifies the loss between predicted value and ground truth.
    It just is squared difference between the two.
    This 'amplification' helps in faster convergence in regression problems as compared
    to just difference between the values - also called Mean Average error.
    '''
    return np.sum(np.square(inp - pred)) #inp: Inputs and pred: predictions from your test model

# Computes error between actual and predicted values
def distance(a, X, pred, values):
    '''
    MSE (look at the function above) quantifies loss between ground truth and predicted values. Now, these
    predictions are based on something called a hypothesis - a function we use
    to hypothesize / estimate the trend. Visualize the data first before 
    engineering a hypothesis function.
    
    '''
    inp = np.exp(a[0] * x) * np.sin(a[1] * x) #<-----Tweak the hypothesis here
    msd = mse(inp, pred) #Error between prediction and hypothesis for values a,b
    values.append([inp]) 
    print ("Variables (a, b):", a ,"Loss:", msd) #Iterates over a,b to find the best loss
    return msd
  
def visualize(optim_step):
    '''
    Function that is used to plot the goodness of fit for each optimization timestep.
    '''
    import matplotlib.pyplot as pyp
    global x, y, values
    pyp.plot(x, y, x, np.array(values[optim_step]).T[:,0])

### 3.2 Trump dataset

In [ ]:
global x, y # Accessible across the notebook
data = np.load('assets/trump.npy')

# y = f(x)
x = data.item(0)['x']
y = data.item(0)['y']

Let's start by visualising the dataset. a.k.a. data exploration

In [ ]:
iplot({"data": [go.Scatter(x=x, y=y)]})

Can you think of a `hypothesis` i.e a mathematical function to approximate Trump's mood swings?

Let's use a traditional optimiser to learn a good function to fit the data.

In [ ]:
import scipy.optimize as opt

# A placeholder to log loss vs. time
global values
values = [] 

# To change the hypothesis, have a look at the function - 'distance'. 
# Minimizes the distance between prediction and expected using conjugate gradient
opt.minimize(distance, (0.0, 0.0), (x, y, values), method='cg') 

### 3.3 Plot the losses

In [ ]:
# Visualize optimization steps
# This only works on a laptop - Google blocks ipywidgets 
_ = interact(visualize,optim_step = widgets.IntSlider(min=0, max=max(0, len(values) - 1), step=10, continuous_update=False))

## 4: Machine learning

In the previous example, we cheated (in a way) as we knew the functional form of the curve $(sin(a_{0}x) \cdot e^{-a_{1}x}))$, and ran a parametric optimisation. 

This section demonstrates a 'magical' hypothesis function that could be used to predict almost anything.

In [ ]:
PDF('assets/exercise2.pdf', 1, 600, 800)

### 4.1 Neural networks

Demostrate logistic regression, feature engineering, shallow and deep neural networks on the [TensorFlow playground](https://playground.tensorflow.org).

### 4.2 Fitting to random data 
Generate a random dataset with two input features: mood swings and blood pressure

In [ ]:
# Generate two random features, and a random output. Is there a model that could fit?
np.random.seed(0)
#Generate two random features, and a random output. Is there a model that could fit?
random_train_input,random_train_output=generate_random(40,400,2)
random_test_input,random_test_output=generate_random(10,400,2)

# Plotting the input features as well as output
iplot([{"x":np.arange(len(random_train_input[0])), "y":random_train_input[0,:,0],"name":"input1"},
      {"x":np.arange(len(random_train_input[0])), "y":random_train_input[0,:,1],"name":"input2"},
      {"x":np.arange(len(random_train_input[0])), "y":random_train_output[0,:],"name":"output"}])

Let's define a simple 4 layer neural network

In [ ]:
from keras.layers import BatchNormalization,Dense,Input,Flatten
from keras.models import Model
import numpy as np

#The function below generates a neural network model with n_layers
def dense_model(n_layers,timesteps=400,features=2):
    '''
    A hello world Machine learning model
    '''
    np.random.seed(1)
    inp=Input(shape=[timesteps,features])
    out=Flatten()(inp)
    for i in range(n_layers):
        out=Dense((42,timesteps)[i==n_layers-1])(out)
    return Model(inp,out)

model = dense_model(4)
model.summary()

Now we want to learn good values for the weights and biases in our neural network. Let's try the same traditional optimisation strategy we used for the curve fitting example.

### 4.3 Optimisation

In [ ]:
# Run optimization using full hessian
# Note: This should take about 2 mins to complete
# You can interrupt the notebook kernel if it takes too long
viz_loss = []
%time opt.minimize(optimize_cg, convert_weights(model, model.get_weights()),(model, random_train_input, random_train_output, viz_loss), method='cg', options={'maxiter': 1})

In [ ]:
# Plot predictions and ground truth values
# Note: zoom in the plot see the differences
iplot([{"x":np.arange(len(random_train_input[0])),"y":model.predict(random_train_input)[0],"name":'predictions'},
      {"x":np.arange(len(random_train_input[0])), "y":random_train_output[0,:],"name":'ground truth'}])

Unfortunately, the optimiser was unable to learn good weights and bias values for the neural network to accurately predict the ground truth. This highlights the limitations of using traditional optimisation methods for training neural networks. 

It should be noted that this example model is very small only comprising of 361 parameters. In practice, it is not uncommon to train neural networks with 100s of thousands to 10s of millions of parameters for complex tasks such as object detection and image segmentation.

### 4.4 Backpropagation
Let's re-train the model using a different optimisation strategy called <b>backpropagation</b> (backprop for short). Backprop has the advantage of optimising (updating) values for a given neural network layer with only its connected neighbours / layers. 

Technically speaking, this causes the hessian to be sparse as parameters are decoupled from other non-neighbouring parameters in the graph which results in faster optimisation.

To illustrate, we can define our neural network as:

$network=f_{1}(\ f_{2} (\ f_{3}(\ f_{4}(x)))))$

where $x$ represents our input data and the function $f_{n}$ represents the _nth_ layer in the network. For example, parameters in the second layer $f_{2}$ only interact with parameters $f_{1}$ and $f_{3}$.


In [ ]:
# Reinitalise the model (i.e. reset weights)
model = dense_model(2)
# Use the adam (ADAptive Momentum) optimisation algorithm 
# with the mse (mean square error) loss function
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model for 100 epochs (number of training iterations through the whole dataset)
# Note: This should take about 1min to complete training
model.fit(random_train_input, random_train_output, 
                validation_data=(random_test_input,random_test_output),
                epochs=400)

In [ ]:
# Plot predictions and ground truth values
# Note: zoom in the plot see the differences
iplot([{"x":np.arange(400),"y":random_train_output[0,:],"name":"input"},
      {"x":np.arange(400),"y":model.predict(random_train_input)[0],"name":"predicted"}])

#replace train with test, what do you see?

There are many backprop optimisation algorithms we can use for training neural networks. Each type has advantages and disadvantages in terms of speed and stability for a given dataset and problem. Some most popular algorithms used for training neural networks are shown below.

<img src='assets/optimizers.gif' width='50%'>
<center>Source: Sebastian Ruder (2018) <a href='http://rudio.io' target='_blank'>http://rudio.io</a></center>

## 5. Convolutions

We trained a simple vanilla neural network (a multi-layer perception - MLP) in the previous section. However, there are other types of neural networks which are better suited for specific types of data and problems. For example, convolutional neural networks (CNNs) are a popular type of network used for image and signal data.

A CNN is comprised of convolution layers. A convolution is a mathematical operation where some input data (e.g. image) is transformed by a convolution matrix of numbers (filter) to generate some output.

In [ ]:
PDF("assets/exercise3.pdf", 1, 600, 800)

Define helper functions used for interactive visualisations 

In [ ]:
# Ignore this function - this is only used to visualize
from PIL import Image # Library to load images
def Infer(Num_filters, Num_layers, max_pool=False):
    global X, parameters, max_pooling
    K.clear_session()
    inp = Input(shape=(788, 1180, 3))
    out = []
    N = Num_layers
    parameters = []
    max_pooling = []
    # Conv-Maxpool - fully connected deck
    for i in range(N):
        out = Conv2D(Num_filters, (3, 3), activation='relu')((out, inp)[i==0])
        if max_pool:
            out = MaxPool2D(2)(out)
    out = Flatten()(out)
    out = Dense(1, activation='sigmoid')(out)

    model = Model(inp,out)
    
    #how many trainable parameters?
    params = int(np.sum([K.count_params(i) for i in set(model.trainable_weights)]))

    #For plotting
    parameters.append(params)
    max_pooling.append(int(max_pool))
    print ("Parameters:", params)
    
    plot_model(model, to_file='assets/model.png')

    return HTML('<img src="assets/model.png?' + str(np.random.randint(0, 1E8)) +' height="240" width="200">')

def Generate_maps(kernel_size=3, stride=1):
    K.clear_session()
    image='assets/lake.jpg'
    img=np.expand_dims(np.array(PIL.Image.open(image)), 0)
    inp=Input(shape=(None, None, 3))
    out=Conv2D(4, int(kernel_size), strides=int(stride), padding='same')(inp)
    out=Conv2D(1, int(kernel_size), strides=int(stride), padding='same')(out)
    model=Model(inp, out)

    return display(Image.fromarray(np.uint8(model.predict(img))[0, :, :, 0]))

### 5.1 Convolutional neural network
To illustrate convolutions, we define a simple two layer CNN that takes a RGB image (3-channels) and outputs a 1-channel image.

In [ ]:
from keras.layers import Conv2D
import keras.backend as K
import PIL

np.random.seed(17)
inp = Input(shape=(None, None, 3))
# conv layer 1: 4 filter, 3 x 3 matrix, default stride of 1
out = Conv2D(4, 3, padding='same')(inp)
# conv layer 2: 1 filter, 3 x 3 matrix, default stride of 1
out = Conv2D(1, 3, padding='same')(out)
model = Model(inp, out)
model.summary()

Show an example image

In [ ]:
Image.open('assets/lake.jpg')

Let's run the image through our CNN model

In [ ]:
#You may ignore the reshaping procedure as PIL.Image expects just 2 dimensions [X,Y] pixels to print the image
img = np.expand_dims(np.array(Image.open('assets/lake.jpg')), 0) 
Image.fromarray(np.uint8(model.predict(img)[0, :, :, 0]))

This is the output of passing the original image through the convolution layers in our network. i.e. The original 3-channel (RGB) input has now been transformed into a 1-channel (single colour) output image.

Note: we have not trained our model on any data but rather used the convolution matrix values that were randomly generated when initialising the model. Surprisingly, these random convolution matrices (filters) were able to do a pretty good job of highlighting (segmenting) the rocks on the left hand side of the image. 

### 5.2 Filters and strides
We can change the number of filters and stride size in each convolution layer. Let's experiment with different values.

In [ ]:
button=widgets.interact_manual(Generate_maps,
                               kernel_size=widgets.IntSlider(min=2, max=10, step=1, continuous_update=False),
                               stride=widgets.IntSlider(min=1, max=10, step=1, continuous_update=False))

Observations:
- increase kernel size = blurier output
- increase stride size = smaller output

### 5.3 Network output
So far we designed a CNN that takes in an image and outputs a transformed image. 

However, for many problems we want our model to output numbers or a category value rather than a transformed image. For example, if developed a CNN to identify tumours in MRI/CT images, we would want our model to output a probability or category (i.e. tumour / no tumour) for a given image. 

These are referred to as classification problems and an example CNN used for image classification is depicted below.

<img src='assets/cnn.png'>
<center>Example of a CNN to classify images</center>

Let's update our model to follow this example CNN architecture ignoring the pooling layers for the moment. 

We will make our model to output the probability of the image belonging to a category by appending two additional layers: `Flatten` and `Dense`.

In [ ]:
from keras.layers import Input, Conv2D, Flatten, Dense
from keras.utils import plot_model

np.random.seed(17)
inp = Input(shape=(img.shape[1], img.shape[2], 3))
# conv layer 1: 4 filter, 3 x 3 matrix, default stride of 1
out = Conv2D(4, 3, padding='same')(inp)
# conv layer 2: 1 filter, 3 x 3 matrix, default stride of 1
out = Conv2D(1, 3, padding='same')(out)
# Flatten the conv layer 2 into a vector of values
# This is required in order to connect the convolution layer with the fully connected layer
out = Flatten()(out)
# Add a fully connected layer with 1 output
# The sigmoid activation function generates a probability score (value between 0 and 1)
out = Dense(1, activation='sigmoid')(out)
model = Model(inp, out)

Run the image through our classification model

In [ ]:
print('Prediction')
print('  probability: %.5f' % (model.predict(img)[0][0]))
print('  category: %d' % (round(model.predict(img)[0][0])))

The predicted probability of the image belonging to category 1 is 0.00006 (0.006%). As such, we classify the image as belong to the other class (category 0).

Let's inspect our model.

In [ ]:
model.summary()

The number of parameters has grown significantly from 149 to 929,990! You can see from the model summary that the Flatten layer is where the majority of new parameters were added.

A `Flatten` layer is required to connect our convolution layers with our fully connected layer that predicts the image category. There are two main issues with designing CNNs with fully connected layers:

1. It can significantly increase the number of parameters in the model 
2. The input data (image) size becomes fixed (e.g. 788 x 1180 in our example)

Note: there are newer variants of CNNs known as Fully Convolutional Networks (FCNs) that replace fully connected layers with convolution layers while still being able to generate classification outputs. FCNs are able to resolve the issues mentioned above at the cost of requiring more compute for running more convolution operations.

### 5.4 Pooling

Pooling layers can be used to reduce the number of parameters in our CNN model by downsampling the outputs of the convolution layers. Max pooling is used as a pooling method in many CNNs is depicted below.

<img src='assets/pooling.png' width='80%'>
<center>Source: MXNet Tutorial, <a href='http://mxnet.io/tutorials/python/mnist.html' target='_blank'>http://mxnet.io/tutorials/python/mnist.html</a></center>

Let's add pooling layers after each convolution layer in our model

In [ ]:
from keras.layers import Input, Conv2D, Flatten, Dense, MaxPool2D
from keras.utils import plot_model

np.random.seed(17)
inp = Input(shape=(img.shape[1], img.shape[2], 3))
# conv layer 1: 4 filter, 3 x 3 matrix, default stride of 1
out = Conv2D(4, 3, padding='same')(inp)
out = MaxPool2D()(out)
# conv layer 2: 1 filter, 3 x 3 matrix, default stride of 1
out = Conv2D(1, 3, padding='same')(out)
out = MaxPool2D()(out)
# Flatten the conv layer 2 into a vector of values
out = Flatten()(out)
# Add a fully connected layer with 1 output
# The sigmoid activation function generates a probability score (value between 0 and 1)
out = Dense(1, activation='sigmoid')(out)
model = Model(inp, out)
model.summary()

The use of pooling layers has signficantly reduced the number of parameters from 929,990 to 58,265 in our classification model.

Experiment the number of filters, layers and use of max pool below to see how the number of parameters change for a given model architecture.

In [ ]:
_=widgets.interact(Infer, Num_filters=widgets.IntSlider(min=2, max=35, step=1, continuous_update=False),
                   Num_layers=widgets.IntSlider(min=1, max=8, step=1, continuous_update=False),
                   max_pool=[('True', True), ('False', False)])

No max pool:
- more filters = more parameters
- more layers = slightly less parameters (more convolutions to pass)

Max pool:
- more filters = more parameters
- more layers = significantly less parameters (more convolutions and pooling to pass)